In [365]:
import torch
from torch import nn,optim
import torch.nn.functional
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os as os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,multilabel_confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc,roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from scipy.stats import kurtosis,skew
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import scipy.stats
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel

import pickle
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE

#import warnings

#warnings.simplefilter(action='ignore', category=FutureWarning)

In [366]:
path_dataframes="/Users/alexdediu/git-projects/Data-Challenge-PHM-Asia-Pacific-2023/dataframes"


In [367]:
df_train = pd.read_csv(f"{path_dataframes}/df_train.csv")
# df_test = pd.read_csv("/Users/alexdediu/git-projects/Data-Challenge-PHM-Asia-Pacific-2023/df_test.csv")

df_labels = pd.read_csv(f"{path_dataframes}/df_labels_train.csv")
df_train = df_train.drop(columns="Unnamed: 0")
df_labels = df_labels.drop(columns="Unnamed: 0")

In [368]:
df_train 

,P1_min,P5_HarmonicAmplitudeRatio,P5_rms,P5_variance,P5_std,P1_median,P1_clearance_factor,P5_min,P1_variance,P1_std,P1_rms,P1_HarmonicAmplitudeRatio,P2_75th_percentile,P2_variance,P2_rms,P2_std,P5_25th_percentile,P2_min,P6_HarmonicAmplitudeRatio,P5_fft_25th_percentile
0,0.151379,0.404716,0.908139,0.989111,0.990348,0.070910,0.835385,0.002855,0.997566,0.997881,0.957028,0.500515,0.793864,0.742221,0.587077,0.764873,0.173932,0.014342,0.535152,0.113235
1,0.151707,0.386983,0.957102,0.970816,0.974082,0.103330,0.856063,0.002856,0.999072,0.999192,0.970137,0.496295,0.762330,0.741918,0.572730,0.764588,0.177174,0.014171,0.534254,0.159739
2,0.152334,0.385712,0.908687,0.942070,0.948400,0.227248,0.884633,0.002595,0.997855,0.998132,0.979246,0.492228,0.765295,0.752047,0.614115,0.774074,0.180487,0.014027,0.516265,0.047297
3,0.152786,0.384052,0.937148,0.964525,0.968475,0.163223,0.884439,0.002830,0.995734,0.996284,0.969917,0.492075,0.775176,0.751956,0.622608,0.773988,0.181395,0.014103,0.509841,0.081392
4,0.152716,0.382267,0.961120,0.975974,0.978674,0.144362,0.873784,0.003023,0.996313,0.996789,0.974559,0.490276,0.792718,0.745000,0.595336,0.767477,0.194015,0.014159,0.505803,0.088930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,0.339054,1.000000,0.205255,0.504254,0.536422,0.048881,0.598945,0.005946,0.585004,0.620029,0.245719,1.000000,0.593336,1.000000,1.000000,1.000000,0.259250,0.087961,0.687747,0.361840
173,0.824787,0.697196,0.719323,0.781425,0.801969,0.509144,0.428516,0.880071,0.583065,0.618157,0.138386,0.717433,0.569406,0.647391,0.448477,0.675020,0.221795,0.040686,0.730663,0.504753
174,0.301860,0.438246,0.344787,0.577233,0.608061,0.772911,0.737837,0.592579,0.531799,0.568273,0.024519,0.362797,0.000000,0.576285,0.348801,0.606348,0.218800,0.026831,0.561968,0.464898
175,0.402989,0.000000,0.204130,0.497790,0.530013,1.000000,0.636050,0.440788,0.500953,0.537907,0.139861,0.000000,0.127847,0.569685,0.239404,0.599916,0.219502,0.024886,0.000000,0.355554


In [369]:
df_labels.head()

,Case,Spacecraft,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1,binary_condition
0,1,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
1,2,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
2,3,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
3,4,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
4,5,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0


In [370]:
column_to_number = {
    'BP1': 1,
    'BP2': 2,
    'BP3': 3,
    'BP4': 4,
    'BP5': 5,
    'BP6': 6,
    'BP7': 7,
    'BV1': 8
}

df_labels['anomaly_location'] = 0

for index, row in df_labels.iterrows():
    for col in column_to_number.keys():
        if row[col] == 1:
            df_labels.at[index, 'anomaly_location'] = column_to_number[col]
            break  

# df_labels = df_labels[153:]

In [371]:
# X = df_train
# y = df_labels['anomaly_location']

# smote = SMOTE(k_neighbors=2,random_state=42)

# X_resampled, y_resampled = smote.fit_resample(X, y)


# df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
# df_resampled['target'] = y_resampled

In [372]:
def feature_selection_anova(df_X,df_y,n_features):
    X = df_X
    y = df_y
    
    f_values, p_values = f_classif(X, y)

    results = pd.DataFrame({'Feature': X.columns, 'F_Value': f_values, 'P_Value': p_values})

    results = results.sort_values(by='F_Value', ascending=False)

    top_50_features = results.head(n_features)
    features = top_50_features["Feature"].values
    # features=np.concatenate([features, ["binary_condition"]])

    print(results["Feature"].values)
    return features

In [373]:
features_train = feature_selection_anova(df_train,df_labels["anomaly_location"],30)
df_train = df_train[features_train]

['P5_HarmonicAmplitudeRatio' 'P1_median' 'P5_min'
 'P1_HarmonicAmplitudeRatio' 'P2_min' 'P6_HarmonicAmplitudeRatio' 'P1_rms'
 'P2_75th_percentile' 'P1_min' 'P2_rms' 'P5_rms' 'P2_variance'
 'P5_variance' 'P2_std' 'P1_variance' 'P5_std' 'P1_clearance_factor'
 'P1_std' 'P5_fft_25th_percentile' 'P5_25th_percentile']


In [374]:
import plotly.graph_objs as go
import plotly.io as pio

# Supponendo che tu abbia già importato i DataFrame df_train e df_labels
# df_train contiene i dati per lo scatter plot
# df_labels contiene la colonna "anomaly_location" che vuoi utilizzare per colorare i punti dello scatter plot

# Lista dei valori unici nella colonna "anomaly_location"
unique_labels = df_labels["anomaly_location"].unique()

# Lista di tracciamenti per ogni valore unico
traces = []

for label in unique_labels:
    # Seleziona i punti corrispondenti a ciascun valore unico
    df_subset = df_train[df_labels["anomaly_location"] == label]
    
    # Crea un tracciamento per i punti con il colore corrispondente e il nome della legenda
    trace = go.Scatter(x=df_subset.index, y=df_subset["P2_variance"], 
                       mode='markers', 
                       marker=dict(color=label),
                       name=str(label))
    
    # Aggiungi il tracciamento alla lista di tracciamenti
    traces.append(trace)

# Aggiungi un secondo scatter plot con tutti i dati
scatter_plot = go.Scatter(x=df_train.index, y=df_train["P1_variance"], 
                          mode='markers', 
                          marker=dict(color='black', size=5), # Aggiungi la dimensione 5 per il secondo scatter plot
                          name='P1_variance')

# Aggiungi il secondo scatter plot alla lista di tracciamenti
traces.append(scatter_plot)

# Layout del grafico con legenda
layout = go.Layout(title="P1_variance - Anomalie", 
                   xaxis=dict(title="Indice"), 
                   yaxis=dict(title="P1_variance"),
                   showlegend=True)

# Crea la figura e visualizzala
fig = go.Figure(data=traces, layout=layout)
pio.show(fig)


In [375]:
df_train

,P5_HarmonicAmplitudeRatio,P1_median,P5_min,P1_HarmonicAmplitudeRatio,P2_min,P6_HarmonicAmplitudeRatio,P1_rms,P2_75th_percentile,P1_min,P2_rms,P5_rms,P2_variance,P5_variance,P2_std,P1_variance,P5_std,P1_clearance_factor,P1_std,P5_fft_25th_percentile,P5_25th_percentile
0,0.404716,0.070910,0.002855,0.500515,0.014342,0.535152,0.957028,0.793864,0.151379,0.587077,0.908139,0.742221,0.989111,0.764873,0.997566,0.990348,0.835385,0.997881,0.113235,0.173932
1,0.386983,0.103330,0.002856,0.496295,0.014171,0.534254,0.970137,0.762330,0.151707,0.572730,0.957102,0.741918,0.970816,0.764588,0.999072,0.974082,0.856063,0.999192,0.159739,0.177174
2,0.385712,0.227248,0.002595,0.492228,0.014027,0.516265,0.979246,0.765295,0.152334,0.614115,0.908687,0.752047,0.942070,0.774074,0.997855,0.948400,0.884633,0.998132,0.047297,0.180487
3,0.384052,0.163223,0.002830,0.492075,0.014103,0.509841,0.969917,0.775176,0.152786,0.622608,0.937148,0.751956,0.964525,0.773988,0.995734,0.968475,0.884439,0.996284,0.081392,0.181395
4,0.382267,0.144362,0.003023,0.490276,0.014159,0.505803,0.974559,0.792718,0.152716,0.595336,0.961120,0.745000,0.975974,0.767477,0.996313,0.978674,0.873784,0.996789,0.088930,0.194015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1.000000,0.048881,0.005946,1.000000,0.087961,0.687747,0.245719,0.593336,0.339054,1.000000,0.205255,1.000000,0.504254,1.000000,0.585004,0.536422,0.598945,0.620029,0.361840,0.259250
173,0.697196,0.509144,0.880071,0.717433,0.040686,0.730663,0.138386,0.569406,0.824787,0.448477,0.719323,0.647391,0.781425,0.675020,0.583065,0.801969,0.428516,0.618157,0.504753,0.221795
174,0.438246,0.772911,0.592579,0.362797,0.026831,0.561968,0.024519,0.000000,0.301860,0.348801,0.344787,0.576285,0.577233,0.606348,0.531799,0.608061,0.737837,0.568273,0.464898,0.218800
175,0.000000,1.000000,0.440788,0.000000,0.024886,0.000000,0.139861,0.127847,0.402989,0.239404,0.204130,0.569685,0.497790,0.599916,0.500953,0.530013,0.636050,0.537907,0.355554,0.219502


In [376]:
df_train

,P5_HarmonicAmplitudeRatio,P1_median,P5_min,P1_HarmonicAmplitudeRatio,P2_min,P6_HarmonicAmplitudeRatio,P1_rms,P2_75th_percentile,P1_min,P2_rms,P5_rms,P2_variance,P5_variance,P2_std,P1_variance,P5_std,P1_clearance_factor,P1_std,P5_fft_25th_percentile,P5_25th_percentile
0,0.404716,0.070910,0.002855,0.500515,0.014342,0.535152,0.957028,0.793864,0.151379,0.587077,0.908139,0.742221,0.989111,0.764873,0.997566,0.990348,0.835385,0.997881,0.113235,0.173932
1,0.386983,0.103330,0.002856,0.496295,0.014171,0.534254,0.970137,0.762330,0.151707,0.572730,0.957102,0.741918,0.970816,0.764588,0.999072,0.974082,0.856063,0.999192,0.159739,0.177174
2,0.385712,0.227248,0.002595,0.492228,0.014027,0.516265,0.979246,0.765295,0.152334,0.614115,0.908687,0.752047,0.942070,0.774074,0.997855,0.948400,0.884633,0.998132,0.047297,0.180487
3,0.384052,0.163223,0.002830,0.492075,0.014103,0.509841,0.969917,0.775176,0.152786,0.622608,0.937148,0.751956,0.964525,0.773988,0.995734,0.968475,0.884439,0.996284,0.081392,0.181395
4,0.382267,0.144362,0.003023,0.490276,0.014159,0.505803,0.974559,0.792718,0.152716,0.595336,0.961120,0.745000,0.975974,0.767477,0.996313,0.978674,0.873784,0.996789,0.088930,0.194015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1.000000,0.048881,0.005946,1.000000,0.087961,0.687747,0.245719,0.593336,0.339054,1.000000,0.205255,1.000000,0.504254,1.000000,0.585004,0.536422,0.598945,0.620029,0.361840,0.259250
173,0.697196,0.509144,0.880071,0.717433,0.040686,0.730663,0.138386,0.569406,0.824787,0.448477,0.719323,0.647391,0.781425,0.675020,0.583065,0.801969,0.428516,0.618157,0.504753,0.221795
174,0.438246,0.772911,0.592579,0.362797,0.026831,0.561968,0.024519,0.000000,0.301860,0.348801,0.344787,0.576285,0.577233,0.606348,0.531799,0.608061,0.737837,0.568273,0.464898,0.218800
175,0.000000,1.000000,0.440788,0.000000,0.024886,0.000000,0.139861,0.127847,0.402989,0.239404,0.204130,0.569685,0.497790,0.599916,0.500953,0.530013,0.636050,0.537907,0.355554,0.219502


In [377]:
# df_labels = df_labels[df_labels["anomaly_location"] != 0]

In [378]:
df_labels

,Case,Spacecraft,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1,binary_condition,anomaly_location
0,1,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0,0
1,2,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0,0
2,3,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0,0
3,4,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0,0
4,5,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,173,3,3,100,100,100,100,0,0,0,1,0,0,0,0,1,4
173,174,3,3,100,100,100,100,0,0,0,0,1,0,0,0,1,5
174,175,3,3,100,100,100,100,0,0,0,0,0,1,0,0,1,6
175,176,3,3,100,100,100,100,0,0,0,0,0,0,1,0,1,7


In [379]:
X_train, X_test, y_train, y_test = train_test_split(df_train, df_labels["anomaly_location"], test_size=0.33, random_state=42)

random_forest = RandomForestClassifier(n_estimators=50, random_state=42)

random_forest.fit(X_train, y_train)

predictions = random_forest.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

conf_matrix = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(y_test, predictions)
print("Classification Report:")
print(class_report)


Accuracy: 0.9322033898305084
Confusion Matrix:
[[52  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2]
 [ 1  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  1]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        52
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           5       0.50      1.00      0.67         1
           6       0.00      0.00      0.00         1
           8       0.33      1.00      0.50         1

    accuracy                           0.93        59
   macro avg       0.40      0.57      0.45        59
weighted avg       0.90      0.93      0.91        59



/Users/alexdediu/anaconda3/envs/pytorch-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/alexdediu/anaconda3/envs/pytorch-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/alexdediu/anaconda3/envs/pytorch-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

